In [1]:
import tensorflow as tf
import tensorflow.keras as keras
import tensorflow.keras.layers as layers
import numpy as np
import pathlib
tf.__version__

'2.0.0'

## 稠密网络
DenseNet是对残差网络的改进，主要的区别是在最后使用相连个不是相加

![Image text](http://zh.d2l.ai/_images/densenet.svg)

    ResNet（左）与DenseNet（右）在跨层连接上的主要区别：使用相加和使用连结
    
DenseNet的主要构建模块是稠密块（dense block）和过渡层（transition layer）。前者定义了输入和输出是如何连结的，后者则用来控制通道数，使之不过大。

### 稠密块

In [3]:
def conv_block(num_chanel):
    '''卷积块 使用归一化-》激活-》卷积的结构'''
    net = keras.Sequential([
        
        layers.BatchNormalization(),
        layers.Activation("relu"),
        layers.Conv2D(num_chanel,kernel_size=3,padding="SAME")
    ])
    return net

In [8]:
class DenseBlock(layers.Layer):
    def __init__(self,num_convs,num_chanel,**kwargs):
        super(DenseBlock,self).__init__(**kwargs)
        self.nets = []
        for i in range(num_convs):
            self.nets.append(conv_block(num_chanel))
    def call(self,x):
        for block in self.nets:
            Y = block(x)
            x = layers.concatenate([x,Y],axis=-1)
        return x
        

In [10]:
dbk = DenseBlock(2,2)
x = tf.random.normal(shape=(1,4,4,3))
dbk(x)

<tf.Tensor: id=498, shape=(1, 4, 4, 7), dtype=float32, numpy=
array([[[[-1.0368147 ,  0.6697251 ,  1.6194909 ,  0.20595467,
           0.75002205, -0.26975372,  0.6110288 ],
         [-1.2829692 , -1.4954864 , -1.3055717 , -0.05915384,
          -0.14918953,  0.36398026, -0.61129034],
         [-0.9239982 ,  1.1946514 , -0.85230935,  0.20345391,
           0.4246419 , -0.62845856, -0.61903536],
         [-1.6430733 ,  0.37067288,  1.2175114 ,  1.0778502 ,
           0.26581043, -0.25803944,  0.12376683]],

        [[-1.5444536 , -0.8612828 , -1.1591899 , -0.74813616,
          -0.07419045, -0.39979878, -1.2058747 ],
         [ 1.5539789 , -0.09261205, -1.4822873 , -0.62113094,
           0.08281277,  0.0432386 , -0.09155516],
         [ 1.0333538 ,  0.02673283, -0.31871417, -0.80178916,
          -0.18240494,  0.33101416, -0.57941014],
         [ 0.6644254 , -1.288338  , -1.3305748 , -0.45774728,
           0.12782294,  0.71323144, -0.5694542 ]],

        [[ 0.7947546 , -0.19459051, -0

### 过渡层
由于每个稠密块都会带来通道数的增加，使用过多则会带来过于复杂的模型。过渡层用来控制模型复杂度。它通过 1×1 卷积层来减小通道数，并使用步幅为2的平均池化层减半高和宽，从而进一步降低模型复杂度。

In [11]:
def transition_block(num_chanel):
    net = keras.Sequential(
        [
            layers.BatchNormalization(),
            layers.Activation("relu"),
            layers.Conv2D(num_chanel,kernel_size=1),
            layers.AveragePooling2D(pool_size=(2,2),strides = 2)
        ]
    )
    return net

In [12]:
tbk = transition_block(10)
out1 = dbk(x)
tbk(out1)

<tf.Tensor: id=619, shape=(1, 2, 2, 10), dtype=float32, numpy=
array([[[[-0.0126456 ,  0.08713236, -0.02637652,  0.20909722,
          -0.11200319,  0.49369764,  0.09739993,  0.18114191,
           0.12171778,  0.40193224],
         [-0.2149685 , -0.00675567, -0.23784779,  0.23309761,
           0.06407173,  0.3626266 , -0.08743507,  0.154962  ,
           0.1982415 ,  0.60913813]],

        [[ 0.0762403 , -0.18736482, -0.36718363,  0.17342336,
          -0.03194648,  0.30088085, -0.03317501,  0.05320933,
           0.27683806,  0.36614457],
         [-0.00628484,  0.0262167 , -0.11096232,  0.02519948,
          -0.01937193,  0.05943681, -0.00436193,  0.07345705,
           0.09532285,  0.07405222]]]], dtype=float32)>

### DenseNet 模型

In [13]:
net = keras.Sequential(
    [
        layers.InputLayer(input_shape=(224,224,3)),
        layers.Conv2D(64,kernel_size=7,strides=3,padding="same"),
        layers.BatchNormalization(),
        layers.Activation("relu"),
        layers.MaxPooling2D(pool_size=(3,3),strides=2,padding="same"),
    ]
)

In [14]:
num_channels, growth_rate = 64, 32  # num_channels为当前的通道数
num_convs_in_dense_blocks = [4, 4, 4, 4]

In [15]:
for i,num_convs in enumerate(num_convs_in_dense_blocks):
    net.add(DenseBlock(num_convs,growth_rate))
    num_channels += num_channels + growth_rate
    
    if i != len(num_convs_in_dense_blocks)-1:
        num_channels = num_channels // 2
        net.add(transition_block(num_channels))

In [16]:
net.add(layers.BatchNormalization())
net.add(layers.Activation("relu"))
net.add(layers.GlobalAveragePooling2D())
net.add(layers.Dense(5,activation="softmax"))

In [17]:
net.summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_7 (Conv2D)            (None, 75, 75, 64)        9472      
_________________________________________________________________
batch_normalization_7 (Batch (None, 75, 75, 64)        256       
_________________________________________________________________
activation_7 (Activation)    (None, 75, 75, 64)        0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 38, 38, 64)        0         
_________________________________________________________________
dense_block_4 (DenseBlock)   (None, 38, 38, 192)       130944    
_________________________________________________________________
sequential_12 (Sequential)   (None, 19, 19, 80)        16208     
_________________________________________________________________
dense_block_5 (DenseBlock)   (None, 19, 19, 208)      

In [18]:
x = tf.random.normal(shape=(1,224,224,3))
net(x)

<tf.Tensor: id=3963, shape=(1, 5), dtype=float32, numpy=
array([[0.19121608, 0.17145461, 0.16664892, 0.18089601, 0.28978434]],
      dtype=float32)>

In [19]:
net.compile(loss=keras.losses.sparse_categorical_crossentropy,metrics=["acc"])

In [20]:
IMG_WIDTH = 224
IMG_HEIGHT = 224
BATCH_SIZE = 5

In [21]:
DATA_URL = "https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz"
data_dir = tf.keras.utils.get_file(fname="flower_photos",origin=DATA_URL,untar=True)
import pathlib
data_dir = pathlib.Path(data_dir)
ds = tf.data.Dataset.list_files(str(data_dir/'*/*'))
class_lable =  np.array([ item.name for item in data_dir.glob("*") if item.name != "LICENSE.txt" ])
## 转换lable为数值型数据
lable_dic =dict( zip(class_lable,np.asarray(range(0,len(class_lable)))))
@tf.function
def get_lable(x):
    arr_str = tf.strings.split(x,sep="\\")
    return arr_str[-2] == class_lable

def decode_img(img):
    img = tf.image.decode_image(img,channels=3)
    img = tf.image.convert_image_dtype(img, tf.float32)
    img = tf.image.resize_with_crop_or_pad(img,IMG_WIDTH,IMG_HEIGHT)
    img = tf.image.random_flip_left_right(img)
    img = tf.image.random_flip_up_down(img)
    return img

def img_map_fun(x):
    lable = get_lable(x)
    img = tf.io.read_file(x)
    img = decode_img(img)
    return img,lable

lable = tf.constant([0,1,2,3,4])
@tf.function
def map_lable_fun(x,y):
    print(y)
    y = lable[y][0]
    return x,y

ds = ds.map(map_func=img_map_fun,num_parallel_calls=tf.data.experimental.AUTOTUNE)
ds = ds.map(map_func=map_lable_fun)

def prepare_for_training(ds, cache=True, shuffle_buffer_size=1000,batch_size = 256):
    # This is a small dataset, only load it once, and keep it in memory.
    # use `.cache(filename)` to cache preprocessing work for datasets that don't
    # fit in memory.
    if cache:
        if isinstance(cache, str):
            ds = ds.cache(cache)
    else:
        ds = ds.cache()

    ds = ds.shuffle(buffer_size=shuffle_buffer_size)

    # Repeat forever
    ds = ds.repeat()

    ds = ds.batch(batch_size)

    # `prefetch` lets the dataset fetch batches in the background while the model
    # is training.
    ds = ds.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

    return ds

ds = prepare_for_training(ds,batch_size = BATCH_SIZE)

image_batch, label_batch = next(iter(ds))

Tensor("y:0", shape=(5,), dtype=bool)


In [22]:
net(image_batch)

<tf.Tensor: id=4981, shape=(5, 5), dtype=float32, numpy=
array([[0.19351912, 0.1949397 , 0.19399604, 0.19807075, 0.21947442],
       [0.19598795, 0.19492997, 0.19351491, 0.1984611 , 0.21710601],
       [0.1974636 , 0.19695452, 0.1944998 , 0.19710729, 0.21397471],
       [0.1953928 , 0.19627978, 0.19487058, 0.19970813, 0.21374863],
       [0.19632544, 0.19590735, 0.1938132 , 0.19858308, 0.21537088]],
      dtype=float32)>

In [26]:
net.fit(ds,steps_per_epoch=100,epochs=10)

Train for 100 steps
Epoch 1/10
100/100 [==============================] - 70s 701ms/step - loss: 1.2935 - acc: 0.4600
Epoch 2/10
100/100 [==============================] - 68s 678ms/step - loss: 1.3254 - acc: 0.4620
Epoch 3/10
100/100 [==============================] - 67s 674ms/step - loss: 1.3471 - acc: 0.4160
Epoch 4/10
100/100 [==============================] - 67s 674ms/step - loss: 1.2814 - acc: 0.4600
Epoch 5/10
100/100 [==============================] - 68s 679ms/step - loss: 1.2745 - acc: 0.4660
Epoch 6/10
100/100 [==============================] - 67s 672ms/step - loss: 1.3460 - acc: 0.4620
Epoch 7/10
100/100 [==============================] - 66s 664ms/step - loss: 1.2660 - acc: 0.4620
Epoch 8/10
100/100 [==============================] - 69s 692ms/step - loss: 1.2113 - acc: 0.5440
Epoch 9/10
100/100 [==============================] - 70s 696ms/step - loss: 1.1773 - acc: 0.5260
Epoch 10/10
100/100 [==============================] - 68s 685ms/step - loss: 1.2816 - acc: 0.5020